# ਏਜੈਂਟਿਕ RAG ਨਾਲ ਆਟੋਜਨ

ਇਹ ਨੋਟਬੁੱਕ ਵਧੇਰੇ ਮੁਲਾਂਕਣ ਸਮਰੱਥਾਵਾਂ ਵਾਲੇ ਆਟੋਜਨ ਏਜੈਂਟਾਂ ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਰੀਟਰੀਵਲ-ਅਗਮੈਂਟਡ ਜਨਰੇਸ਼ਨ (RAG) ਨੂੰ ਲਾਗੂ ਕਰਨ ਦਾ ਪ੍ਰਦਰਸ਼ਨ ਕਰਦੀ ਹੈ।


SQLite ਵਰਜਨ ਦੀ ਸਮੱਸਿਆ ਠੀਕ ਕਰੋ  
ਜੇ ਤੁਸੀਂ ਇਹ ਗਲਤੀ ਵੇਖੋ:  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```  

ਆਪਣੇ ਨੋਟਬੁੱਕ ਦੇ ਸ਼ੁਰੂ ਵਿੱਚ ਇਸ ਕੋਡ ਬਲਾਕ ਨੂੰ ਅਨਕਮੈਂਟ ਕਰੋ:  


In [2]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [3]:
import os
import time
import asyncio
from typing import List, Dict
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core import CancellationToken
from autogen_agentchat.messages import TextMessage
from azure.core.credentials import AzureKeyCredential
from autogen_ext.models.azure import AzureAIChatCompletionClient

import chromadb

load_dotenv()

True

## ਕਲਾਇੰਟ ਬਣਾਓ

ਸਭ ਤੋਂ ਪਹਿਲਾਂ, ਅਸੀਂ Azure AI Chat Completion Client ਨੂੰ ਸ਼ੁਰੂ ਕਰਦੇ ਹਾਂ। ਇਹ ਕਲਾਇੰਟ ਵਰਤੋਂਕਾਰ ਦੇ ਪ੍ਰਸ਼ਨਾਂ ਦੇ ਜਵਾਬ ਤਿਆਰ ਕਰਨ ਲਈ Azure OpenAI ਸੇਵਾ ਨਾਲ ਸੰਚਾਰ ਕਰਨ ਲਈ ਵਰਤਿਆ ਜਾਵੇਗਾ।


In [4]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

## ਵੇਕਟਰ ਡੇਟਾਬੇਸ ਸ਼ੁਰੂਆਤ

ਅਸੀਂ ChromaDB ਨੂੰ ਸਥਾਈ ਸਟੋਰੇਜ ਨਾਲ ਸ਼ੁਰੂ ਕਰਦੇ ਹਾਂ ਅਤੇ ਵਧੇਰੇ ਨਮੂਨਾ ਦਸਤਾਵੇਜ਼ ਸ਼ਾਮਲ ਕਰਦੇ ਹਾਂ। ChromaDB ਨੂੰ ਉਹ ਦਸਤਾਵੇਜ਼ ਸਟੋਰ ਅਤੇ ਰੀਟਰੀਵ ਕਰਨ ਲਈ ਵਰਤਿਆ ਜਾਵੇਗਾ ਜੋ ਸਹੀ ਜਵਾਬ ਤਿਆਰ ਕਰਨ ਲਈ ਸੰਦਰਭ ਪ੍ਰਦਾਨ ਕਰਦੇ ਹਨ।


In [5]:
# Initialize ChromaDB with persistent storage
chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.create_collection(
    name="travel_documents",
    metadata={"description": "travel_service"},
    get_or_create=True
)

# Enhanced sample documents
documents = [
    "Contoso Travel offers luxury vacation packages to exotic destinations worldwide.",
    "Our premium travel services include personalized itinerary planning and 24/7 concierge support.",
    "Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage.",
    "Popular destinations include the Maldives, Swiss Alps, and African safaris.",
    "Contoso Travel provides exclusive access to boutique hotels and private guided tours."
]

# Add documents with metadata
collection.add(
    documents=documents,
    ids=[f"doc_{i}" for i in range(len(documents))],
    metadatas=[{"source": "training", "type": "explanation"} for _ in documents]
)

## ਕੌਂਟੈਕਸਟ ਪ੍ਰੋਵਾਈਡਰ ਇੰਪਲੀਮੈਂਟੇਸ਼ਨ

`ContextProvider` ਕਲਾਸ ਕਈ ਸਰੋਤਾਂ ਤੋਂ ਕੌਂਟੈਕਸਟ ਪ੍ਰਾਪਤੀ ਅਤੇ ਇੰਟੀਗ੍ਰੇਸ਼ਨ ਨੂੰ ਸੰਭਾਲਦੀ ਹੈ:

1. **ਵੇਕਟਰ ਡੇਟਾਬੇਸ ਰੀਟਰੀਵਲ**: ChromaDB ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਯਾਤਰਾ ਦਸਤਾਵੇਜ਼ਾਂ 'ਤੇ ਸੈਮਾਂਟਿਕ ਖੋਜ ਕਰਦੀ ਹੈ
2. **ਮੌਸਮ ਜਾਣਕਾਰੀ**: ਵੱਡੇ ਸ਼ਹਿਰਾਂ ਲਈ ਇੱਕ ਨਕਲੀ ਮੌਸਮ ਡੇਟਾਬੇਸ ਨੂੰ ਸੰਭਾਲਦੀ ਹੈ
3. **ਇਕਜੁਟ ਕੌਂਟੈਕਸਟ**: ਦਸਤਾਵੇਜ਼ ਅਤੇ ਮੌਸਮ ਡੇਟਾ ਦੋਵਾਂ ਨੂੰ ਜੋੜ ਕੇ ਵਿਸਤ੍ਰਿਤ ਕੌਂਟੈਕਸਟ ਤਿਆਰ ਕਰਦੀ ਹੈ

ਮੁੱਖ ਮੈਥਡ:
- `get_retrieval_context()`: ਪ੍ਰਸ਼ਨ ਦੇ ਆਧਾਰ 'ਤੇ ਸਬੰਧਤ ਦਸਤਾਵੇਜ਼ ਪ੍ਰਾਪਤ ਕਰਦੀ ਹੈ
- `get_weather_data()`: ਨਿਰਧਾਰਤ ਸਥਾਨਾਂ ਲਈ ਮੌਸਮ ਜਾਣਕਾਰੀ ਪ੍ਰਦਾਨ ਕਰਦੀ ਹੈ
- `get_unified_context()`: ਦਸਤਾਵੇਜ਼ ਅਤੇ ਮੌਸਮ ਕੌਂਟੈਕਸਟ ਨੂੰ ਜੋੜ ਕੇ ਬਿਹਤਰ ਜਵਾਬਾਂ ਲਈ ਤਿਆਰ ਕਰਦੀ ਹੈ


In [6]:
class ContextProvider:
    def __init__(self, collection):
        self.collection = collection
        # Simulated weather database
        self.weather_database = {
            "new york": {"temperature": 72, "condition": "Partly Cloudy", "humidity": 65, "wind": "10 mph"},
            "london": {"temperature": 60, "condition": "Rainy", "humidity": 80, "wind": "15 mph"},
            "tokyo": {"temperature": 75, "condition": "Sunny", "humidity": 50, "wind": "5 mph"},
            "sydney": {"temperature": 80, "condition": "Clear", "humidity": 45, "wind": "12 mph"},
            "paris": {"temperature": 68, "condition": "Cloudy", "humidity": 70, "wind": "8 mph"},
        }
    
    def get_retrieval_context(self, query: str) -> str:
        """Retrieves relevant documents from vector database based on query."""
        results = self.collection.query(
            query_texts=[query],
            include=["documents", "metadatas"],
            n_results=2
        )
        context_strings = []
        if results and results.get("documents") and len(results["documents"][0]) > 0:
            for doc, meta in zip(results["documents"][0], results["metadatas"][0]):
                context_strings.append(f"Document: {doc}\nMetadata: {meta}")
        return "\n\n".join(context_strings) if context_strings else "No relevant documents found"
    
    def get_weather_data(self, location: str) -> str:
        """Simulates retrieving weather data for a given location."""
        if not location:
            return ""
            
        location_key = location.lower()
        if location_key in self.weather_database:
            data = self.weather_database[location_key]
            return f"Weather for {location.title()}:\n" \
                   f"Temperature: {data['temperature']}°F\n" \
                   f"Condition: {data['condition']}\n" \
                   f"Humidity: {data['humidity']}%\n" \
                   f"Wind: {data['wind']}"
        else:
            return f"No weather data available for {location}."
    
    def get_unified_context(self, query: str, location: str = None) -> str:
        """Returns a unified context combining both document retrieval and weather data."""
        retrieval_context = self.get_retrieval_context(query)
        
        weather_context = ""
        if location:
            weather_context = self.get_weather_data(location)
            weather_intro = f"\nWeather Information for {location}:\n"
        else:
            weather_intro = ""
        
        return f"Retrieved Context:\n{retrieval_context}\n\n{weather_intro}{weather_context}"

## ਏਜੰਟ ਸੰਰਚਨਾ

ਅਸੀਂ ਰਿਟਰੀਵਲ ਅਤੇ ਸਹਾਇਕ ਏਜੰਟਾਂ ਦੀ ਸੰਰਚਨਾ ਕਰਦੇ ਹਾਂ। ਰਿਟਰੀਵਲ ਏਜੰਟ ਸੈਮੈਂਟਿਕ ਖੋਜ ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਸਬੰਧਤ ਜਾਣਕਾਰੀ ਲੱਭਣ ਵਿੱਚ ਮਾਹਰ ਹੈ, ਜਦਕਿ ਸਹਾਇਕ ਲੱਭੀ ਗਈ ਜਾਣਕਾਰੀ ਦੇ ਆਧਾਰ 'ਤੇ ਵਿਸਥਾਰਪੂਰਵਕ ਜਵਾਬ ਤਿਆਰ ਕਰਦਾ ਹੈ।


In [7]:
# Create agents with enhanced capabilities
assistant = AssistantAgent(
    name="assistant",
    model_client=client,
    system_message=(
        "You are a helpful AI assistant that provides answers using ONLY the provided context. "
        "Do NOT include any external information. Base your answer entirely on the context given below."
    ),
)

## ਰਾਗ ਨਾਲ ਪ੍ਰਸ਼ਨ ਪ੍ਰਕਿਰਿਆ

ਅਸੀਂ `ask_rag` ਫੰਕਸ਼ਨ ਨੂੰ ਸਥਾਪਿਤ ਕਰਦੇ ਹਾਂ ਤਾਂ ਜੋ ਸਹਾਇਕ ਨੂੰ ਪ੍ਰਸ਼ਨ ਭੇਜਿਆ ਜਾ ਸਕੇ, ਜਵਾਬ ਦੀ ਪ੍ਰਕਿਰਿਆ ਕੀਤੀ ਜਾ ਸਕੇ, ਅਤੇ ਇਸਦੀ ਮੂਲਾਂਕਣ ਕੀਤੀ ਜਾ ਸਕੇ। ਇਹ ਫੰਕਸ਼ਨ ਸਹਾਇਕ ਨਾਲ ਸੰਚਾਰ ਨੂੰ ਸੰਭਾਲਦਾ ਹੈ ਅਤੇ ਜਵਾਬ ਦੀ ਗੁਣਵੱਤਾ ਨੂੰ ਮਾਪਣ ਲਈ ਮੂਲਾਂਕਣਕਰਤਾ ਦੀ ਵਰਤੋਂ ਕਰਦਾ ਹੈ।


In [8]:
async def ask_rag_agent(query: str, context_provider: ContextProvider, location: str = None):
    """
    Sends a query to the assistant agent with context from the provider.
    
    Args:
        query: The user's question
        context_provider: The context provider instance
        location: Optional location for weather queries
    """
    try:
        # Get unified context
        context = context_provider.get_unified_context(query, location)
        
        # Augment the query with context
        augmented_query = (
            f"{context}\n\n"
            f"User Query: {query}\n\n"
            "Based ONLY on the above context, please provide a helpful answer."
        )

        # Send the augmented query to the assistant
        start_time = time.time()
        response = await assistant.on_messages(
            [TextMessage(content=augmented_query, source="user")],
            cancellation_token=CancellationToken(),
        )
        processing_time = time.time() - start_time
        
        return {
            'query': query,
            'response': response.chat_message.content,
            'processing_time': processing_time,
            'location': location
        }
    except Exception as e:
        print(f"Error processing query: {e}")
        return None

# ਉਦਾਹਰਨ ਵਜੋਂ ਵਰਤੋਂ

ਅਸੀਂ evaluator ਨੂੰ ਸ਼ੁਰੂ ਕਰਦੇ ਹਾਂ ਅਤੇ ਉਹ ਕੁਝ ਪ੍ਰਸ਼ਨ ਪਰਿਭਾਸ਼ਿਤ ਕਰਦੇ ਹਾਂ ਜਿਨ੍ਹਾਂ ਨੂੰ ਅਸੀਂ ਪ੍ਰਕਿਰਿਆ ਕਰਨੀ ਅਤੇ ਮੁਲਾਂਕਣ ਕਰਨਾ ਚਾਹੁੰਦੇ ਹਾਂ।


In [ ]:
async def main():
    # Initialize context provider
    context_provider = ContextProvider(collection)
    
    # Example queries
    queries = [
        {"query": "What does Contoso's travel insurance cover?"},
        {"query": "What's the weather like in London?", "location": "london"},
        {"query": "What luxury destinations does Contoso offer and what's the weather in Paris?", "location": "paris"},
    ]
    
    print("=== Autogen RAG Demo ===")
    for query_data in queries:
        query = query_data["query"]
        location = query_data.get("location")
        
        print(f"\n\nQuery: {query}")
        if location:
            print(f"Location: {location}")
        
        # Show the context being used
        context = context_provider.get_unified_context(query, location)
        print("\n--- Context Used ---")
        print(context)
        print("-------------------")
        
        # Get response from the agent
        result = await ask_rag_agent(query, context_provider, location)
        if result:
            print(f"\nResponse: {result['response']}")
        print("\n" + "="*50)

## ਸਕ੍ਰਿਪਟ ਚਲਾਓ

ਅਸੀਂ ਜਾਂਚਦੇ ਹਾਂ ਕਿ ਸਕ੍ਰਿਪਟ ਇੰਟਰਐਕਟਿਵ ਵਾਤਾਵਰਣ ਵਿੱਚ ਚੱਲ ਰਿਹਾ ਹੈ ਜਾਂ ਇੱਕ ਸਧਾਰਣ ਸਕ੍ਰਿਪਟ ਵਜੋਂ, ਅਤੇ ਮੁੱਖ ਫੰਕਸ਼ਨ ਨੂੰ ਉਸ ਅਨੁਸਾਰ ਚਲਾਉਂਦੇ ਹਾਂ।


In [12]:
if __name__ == "__main__":
    if asyncio.get_event_loop().is_running():
        await main()
    else:
        asyncio.run(main())

=== Autogen RAG Demo ===


Query: What does Contoso's travel insurance cover?

--- Context Used ---
Retrieved Context:
Document: Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage.
Metadata: {'source': 'training', 'type': 'explanation'}

Document: Contoso Travel offers luxury vacation packages to exotic destinations worldwide.
Metadata: {'source': 'training', 'type': 'explanation'}


-------------------

Response: Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage.



Query: What's the weather like in London?
Location: london

--- Context Used ---
Retrieved Context:
Document: Popular destinations include the Maldives, Swiss Alps, and African safaris.
Metadata: {'source': 'training', 'type': 'explanation'}

Document: Contoso Travel offers luxury vacation packages to exotic destinations worldwide.
Metadata: {'source': 'training', 'type': 'explanation'}


Weather Information for london:
Weather for London:
T


---

**ਅਸਵੀਕਰਤੀ**:  
ਇਹ ਦਸਤਾਵੇਜ਼ AI ਅਨੁਵਾਦ ਸੇਵਾ [Co-op Translator](https://github.com/Azure/co-op-translator) ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਅਨੁਵਾਦ ਕੀਤਾ ਗਿਆ ਹੈ। ਜਦੋਂ ਕਿ ਅਸੀਂ ਸਹੀ ਹੋਣ ਦਾ ਯਤਨ ਕਰਦੇ ਹਾਂ, ਕਿਰਪਾ ਕਰਕੇ ਧਿਆਨ ਦਿਓ ਕਿ ਸਵੈਚਾਲਿਤ ਅਨੁਵਾਦਾਂ ਵਿੱਚ ਗਲਤੀਆਂ ਜਾਂ ਅਸੁੱਤੀਆਂ ਹੋ ਸਕਦੀਆਂ ਹਨ। ਇਸ ਦੀ ਮੂਲ ਭਾਸ਼ਾ ਵਿੱਚ ਮੌਜੂਦ ਮੂਲ ਦਸਤਾਵੇਜ਼ ਨੂੰ ਪ੍ਰਮਾਣਿਕ ਸਰੋਤ ਮੰਨਿਆ ਜਾਣਾ ਚਾਹੀਦਾ ਹੈ। ਮਹੱਤਵਪੂਰਨ ਜਾਣਕਾਰੀ ਲਈ, ਪੇਸ਼ੇਵਰ ਮਨੁੱਖੀ ਅਨੁਵਾਦ ਦੀ ਸਿਫਾਰਸ਼ ਕੀਤੀ ਜਾਂਦੀ ਹੈ। ਇਸ ਅਨੁਵਾਦ ਦੀ ਵਰਤੋਂ ਤੋਂ ਪੈਦਾ ਹੋਣ ਵਾਲੇ ਕਿਸੇ ਵੀ ਗਲਤਫਹਿਮੀ ਜਾਂ ਗਲਤ ਵਿਆਖਿਆ ਲਈ ਅਸੀਂ ਜ਼ਿੰਮੇਵਾਰ ਨਹੀਂ ਹਾਂ।  
